In [67]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:

df = pd.read_csv('reduced_Reviews.csv')
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,165257,B000EVG8J2,A1L01D2BD3RKVO,"B. Miller ""pet person""",0,0,5,1268179200,Crunchy & Good Gluten-Free Sandwich Cookies!,Having tried a couple of other brands of glute...
1,231466,B0000BXJIS,A3U62RE5XZDP0G,Marty,0,0,5,1298937600,great kitty treats,My cat loves these treats. If ever I can't fin...
2,427828,B008FHUFAU,AOXC0JQQZGGB6,Kenneth Shevlin,0,2,3,1224028800,COFFEE TASTE,A little less than I expected. It tends to ha...
3,433955,B006BXV14E,A3PWPNZVMNX3PA,rareoopdvds,0,1,2,1335312000,So the Mini-Wheats were too big?,"First there was Frosted Mini-Wheats, in origin..."
4,70261,B007I7Z3Z0,A1XNZ7PCE45KK7,Og8ys1,0,2,5,1334707200,Great Taste . . .,and I want to congratulate the graphic artist ...
...,...,...,...,...,...,...,...,...,...,...
4995,370837,B003NZHOQ2,AAFMY4T753D99,markpittsnh,2,3,3,1312329600,Amazing bar! Too expensive @ amazon!,Why is the box of 10 $10 @ Amazon when I can b...
4996,135373,B005VOOM4A,A3H9JSM1SUTE4O,Robin Simmons,1,3,4,1337212800,BOB MARLEY WOULD LIKE THIS I THINK,"I liked the smooth, rather mild flavor of this..."
4997,268329,B0040K41MY,A37PZJH2F13IOR,"J. Rose ""JR From the Ball so Blue""",0,0,5,1340841600,"Tasty, delicious start to your day",Nature's Path is a brand I know and trust. Ver...
4998,29455,B000PDY3P0,A2Q1NU40QODAXT,Tony H,0,0,5,1308614400,Tastes great!,"I have a Waring Pro, and this stuff is fantast..."


In [69]:
import pandas as pd
import re
df = pd.read_csv("reduced_Reviews.csv")

df = df.dropna(subset=['Text', 'Summary'])

In [70]:
df = df.dropna(subset=['Text', 'Summary'])

def clean(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text


df['cleaned_text'] = df['Text'].apply(clean)
df['cleaned_summary'] = df['Summary'].apply(clean)



print(df[['cleaned_text', 'cleaned_summary']].head())


                                        cleaned_text  \
0  having tried a couple of other brands of glute...   
1  my cat loves these treats if ever i cant find ...   
2  a little less than i expected  it tends to hav...   
3  first there was frosted miniwheats in original...   
4  and i want to congratulate the graphic artist ...   

                             cleaned_summary  
0  crunchy  good glutenfree sandwich cookies  
1                         great kitty treats  
2                               coffee taste  
3             so the miniwheats were too big  
4                             great taste     


In [71]:
from statistics import mode

# Use the cleaned text and summary data from the DataFrame
input_words = sorted(list(set(df['cleaned_text'])))  # Changed 'texts' to 'df['cleaned_text']'
target_words = sorted(list(set(df['cleaned_summary']))) # Changed 'summaries' to 'df['cleaned_summary']'

num_inwords = len(input_words)
num_tagwords = len(target_words)
maxinput = mode([len(i) for i in df['cleaned_text']]) # Calculate maximum input length
maxtlen = mode([len(i) for i in df['cleaned_summary']]) # Calculate maximum target length


print("no. of input words :", num_inwords)
print("no. of target words :", num_tagwords)
print("maximum input length :", maxinput) # Changed 'max_text_length' to 'maxinput'
print("maximum target length :", maxtlen) # Changed 'max_summary_length' to 'maxtlen'

no. of input words : 4944
no. of target words : 4245
maximum input length : 148
maximum target length : 9


In [72]:
MAX_TEXT_LEN = 148
MAX_SUMMARY_LEN = 9

df = df[df['cleaned_text'].apply(lambda x: len(x.split()) <= MAX_TEXT_LEN)]
df = df[df['cleaned_summary'].apply(lambda x: len(x.split()) <= MAX_SUMMARY_LEN)]

In [73]:
max_text_len = 148
max_summary_len = 9
text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(df['cleaned_text'])
text_sequences = text_tokenizer.texts_to_sequences(df['cleaned_text'])
text_padded = pad_sequences(text_sequences, maxlen=max_text_len, padding='post')


summary_tokenizer = Tokenizer()
summary_tokenizer.fit_on_texts(df['cleaned_summary'])
summary_sequences = summary_tokenizer.texts_to_sequences(df['cleaned_summary'])
summary_padded = pad_sequences(summary_sequences, maxlen=max_summary_len, padding='post')

In [74]:
# Tokenize inputs
text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(df['cleaned_text'])
text_seq = text_tokenizer.texts_to_sequences(df['cleaned_text'])

summary_tokenizer = Tokenizer(oov_token='<unk>')
summary_tokenizer.fit_on_texts(['start ' + text + ' end' for text in df['cleaned_summary']])
summary_seq = summary_tokenizer.texts_to_sequences(['start ' + text + ' end' for text in df['cleaned_summary']])

encoder_input = pad_sequences(text_seq, maxlen=MAX_TEXT_LEN, padding='post')
decoder_input = pad_sequences(summary_seq, maxlen=MAX_SUMMARY_LEN, padding='post')

In [75]:

text_vocab_size = len(text_tokenizer.word_index)
summary_vocab_size = len(summary_tokenizer.word_index)

print("Text vocab size:", text_vocab_size)
print("Summary vocab size:", summary_vocab_size)


Text vocab size: 13212
Summary vocab size: 2959


In [76]:


decoder_target = np.zeros((len(decoder_input), MAX_SUMMARY_LEN, len(summary_tokenizer.word_index)+1))
for i, seq in enumerate(decoder_input):
    for t in range(1, len(seq)):
        decoder_target[i, t-1, seq[t]] = 1.0


encoder_vocab = len(text_tokenizer.word_index) + 1
decoder_vocab = len(summary_tokenizer.word_index) + 1

In [77]:

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense


latent_dim = 128

encoder_inputs = Input(shape=(MAX_TEXT_LEN,))
enc_emb = Embedding(encoder_vocab, 148, trainable=True)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

In [78]:


decoder_inputs = Input(shape=(MAX_SUMMARY_LEN,))
dec_emb = Embedding(decoder_vocab, 148, trainable=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(decoder_vocab, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
history = model.fit([encoder_input, decoder_input], decoder_target, batch_size=64, epochs=50, validation_split=0.2 )




Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 35s 582ms/step - accuracy: 0.4476 - loss: 5.6623 - val_accuracy: 0.4986 - val_loss: 3.1200
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 45s 667ms/step - accuracy: 0.5241 - loss: 3.0423 - val_accuracy: 0.6096 - val_loss: 2.9432
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 37s 591ms/step - accuracy: 0.5980 - loss: 2.9306 - val_accuracy: 0.6209 - val_loss: 2.8957
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 29s 543ms/step - accuracy: 0.6147 - loss: 2.8302 - val_accuracy: 0.6212 - val_loss: 2.8741
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 541ms/step - accuracy: 0.6121 - loss: 2.7986 - val_accuracy: 0.6212 - val_loss: 2.8589
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 551ms/step - accuracy: 0.6125 - loss: 2.7494 - val_accuracy: 0.6214 - val_loss: 2.8430
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 28s 525ms/step - accuracy: 0.6137 - loss: 2.7302 - val_accuracy: 0.6219 - val_loss: 2.8335
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 43s 568ms/step - accuracy: 0.6119 - loss: 2.7153 - val_accu

In [ ]:

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(decoder_vocab, 148, trainable=True)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

reverse_target_word_index = summary_tokenizer.index_word
target_word_index = summary_tokenizer.word_index

def decode_sequence(input_seq):

    states_value = encoder_model.predict(input_seq)



In [ ]:

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index.get(sampled_token_index, '<unk>') # Handle unknown words


        if sampled_word == 'end' or len(decoded_sentence.split()) >= (MAX_SUMMARY_LEN - 1):
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word


        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

In [ ]:


num_samples = 5

for i in range(num_samples):
    sample_input = encoder_input[i].reshape(1, MAX_TEXT_LEN)
    print("Original Review:\n", df['Text'].iloc[i])
    print("\nActual Summary:\n", df['cleaned_summary'].iloc[i])
    print("\nPredicted Summary:\n", decode_sequence(sample_input))






In [60]:

from transformers import pipeline
import pandas as pd
import re


dataset_path = "/content/reduced_Reviews.csv"
data = pd.read_csv(dataset_path)

In [61]:

def clean_text(text):
    """Clean and preprocess text data"""
    if not isinstance(text, str):
        return ""

    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9.,!?\'" ]', ' ', text)
    return text.strip()


data['Cleaned_Text'] = data['Text'].apply(clean_text)

summarizer = pipeline(
    "summarization",
    model="t5-base",
    tokenizer="t5-base"
)

Device set to use cpu


In [64]:

def generate_summary_for_index(index):
    """Generate and display summary for text at given index"""
    try:
        row = data.iloc[index]
        print(f"\nOriginal Text (Index {index}, {len(row['Text'].split())} words):")
        print(row['Text'][:500] + ("..." if len(row['Text']) > 500 else ""))

        summary = summarizer(
            row['Cleaned_Text'],
            min_length=10,
            max_length=50,
            do_sample=False
        )

        print(f"\nGenerated Summary ({len(summary[0]['summary_text'].split())} words):")
        print(summary[0]['summary_text'])

        if 'Summary' in data.columns:
            print(f"\nReference Summary ({len(row['Summary'].split())} words):")
            print(row['Summary'])

        print("-" * 80)
    except IndexError:
        print(f"Error: Index {index} is out of range. Dataset has {len(data)} rows.")
    except Exception as e:
        print(f"Error generating summary: {e}")

In [65]:

while True:
    print(f"\nDataset contains {len(data)} rows (indices 0-{len(data)-1})")
    user_input = input("Enter index to summarize (or 'q' to quit): ")

    if user_input.lower() == 'q':
        break

    try:
        index = int(user_input)
        generate_summary_for_index(index)
    except ValueError:
        print("Please enter a valid number or 'q' to quit")



Dataset contains 5000 rows (indices 0-4999)
Enter index to summarize (or 'q' to quit): 4

Original Text (Index 4, 122 words):
and I want to congratulate the graphic artist for putting the entire product name on such a small box.  The ad men must have really thought long and hard.<br /><br />But seriously, I love the product. The taste was refreshing and I thought that the taste was pleasing with no aftertaste.  Not too sweet and Goldilocks would have stopped right there if there was a choice of 3.  Easy to use as you just pour the contents into a 16 oz bottle of water and shake.  Mixed well, no granulation, you really ...

Generated Summary (27 words):
the taste was refreshing and pleasant with no aftertaste . easy to use as you just pour the contents into a 16 oz bottle of water .

Reference Summary (5 words):
Great Taste . . .
--------------------------------------------------------------------------------

Dataset contains 5000 rows (indices 0-4999)
Enter index to summarize (or 'q'